In [1]:
from KGNET import KGNET

/home/hussein/anaconda3/envs/ogb/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/hussein/anaconda3/envs/ogb/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
kgnet=KGNET(KG_endpointUrl='http://206.12.98.118:8890/sparql',KG_NamedGraph_IRI='https://dblp2022.org',KG_Prefix='dblp2022')

## visualize graph schema

In [3]:
types_df=kgnet.getKGNodeEdgeTypes(write_to_file=True,prefix='dblp2022')
types_df=types_df[(~types_df["object"].str.endswith("_Obj")) | (types_df["object"].str.endswith("publishedIn_Obj"))]
g=kgnet.visualizeKG(types_df,Notebook=True,
                    width="100%",height="300px")
g.show('nx.html')

nx.html


## Training Pipeline

In [4]:
taskId,modelId,trainMetadata=kgnet.train_GML(operatorType=KGNET.GML_Operator_Types.NodeClassification,
                                             GNNMethod=KGNET.GNN_Methods.RGCN,
                                             targetNodeType="dblp2022:Publication",labelNodeType="dblp2022:publishedIn_Obj")

targetEdge= https://dblp.org/rdf/schema#publishedIn
sparql_ml_insert_query=  prefix dblp2022:<https://dblp2022.org> 
 prefix kgnet:<https://www.kgnet.com/>
           Insert into <kgnet>
           where{
               select * from kgnet.TrainGML(
               {
"name":"NodeClassifier>dblp2022>dblp2022:Publication>dblp2022:publishedIn_Obj>RGCN",
"GMLTask":{"taskType":"kgnet:NodeClassifier",
"targetNode":"dblp2022:Publication",
"labelNode":"dblp2022:publishedIn_Obj",
"namedGraphURI":"https://dblp2022.org",
"namedGraphPrefix":"dblp2022",
"targetEdge":"https://dblp.org/rdf/schema#publishedIn","GNNMethod":"RGCN",
"datasetTypesFilePath":"//media/hussein/UbuntuData/GithubRepos/KGNET/Datasets/dblp2022_Types.csv",
"TOSG":"d1h1"}
})}
################# TOSG Sampling ###########################
KG' path= //media/hussein/UbuntuData/GithubRepos/KGNET/Datasets/mid-0000099.tsv
rows_count= 868117 Query Time= 0:00:04.810732  sec.
query idx= 0 batches_count= 9
Query idx:  1.0 / 9  records count= 100

virtual memory GB: 9.834426879882812  percent 66.3
Run: 01, Epoch: 01, Loss: 6.9595, Train: 22.53%, Valid: 23.11% Test: 21.91%
Run: 01, Epoch: 02, Loss: 6.2478, Train: 22.51%, Valid: 23.11% Test: 21.91%
Run: 01, Epoch: 03, Loss: 5.5247, Train: 22.55%, Valid: 23.11% Test: 21.91%
Run: 01, Epoch: 04, Loss: 4.7154, Train: 22.57%, Valid: 23.11% Test: 21.91%
Run: 01, Epoch: 05, Loss: 4.4305, Train: 23.78%, Valid: 23.88% Test: 22.98%
Run 01:
Highest Train: 23.78
Highest Valid: 23.88
  Final Train: 23.78
   Final Test: 22.98
model run  0  train time CPU= 0:00:45.156429  sec.
resource.struct_rusage(ru_utime=287.399212, ru_stime=52.082328, ru_maxrss=6452640, ru_ixrss=0, ru_idrss=0, ru_isrss=0, ru_minflt=25407286, ru_majflt=80, ru_nswap=0, ru_inblock=6056, ru_oublock=352, ru_msgsnd=0, ru_msgrcv=0, ru_nsignals=0, ru_nvcsw=378975, ru_nivcsw=5826)
All runs:
Highest Train: 23.78 ± nan
Highest Valid: 23.88 ± nan
  Final Train: 23.78 ± nan
   Final Test: 22.98 ± nan
All runs:
Highest Train: 23.78 ± nan

### Query task Info from KGMeta

In [5]:
kgnet.KGMeta_Governer.getGMLTaskBasicInfoByID(taskId)

,property,Value
0,Task ID,25
1,Task Name,NodeClassifier>dblp2022>dblp2022:Publication>d...
2,Task Type,kgnet:type/nodeClassification
3,KG name,dblp2022
4,KG description,Subgraph of dblp 2022-03 KG that contains only...
5,Target Edge,https://dblp.org/rdf/schema#publishedIn
6,Target Node Type,dblp2022:Publication
7,Date Created,"19/08/2023, 14:46:30"
8,# Trained Models,11
9,Trained Models IDs,"kgnet:GMLModel/mid-0000083,kgnet:GMLModel/mid-..."


### Query Task Trained Models Info from KGMeta
<li> The green color indicates the best model to choose.</li>
<li> The orange color indicates the model selected by KGNET Inference query optimizer which balance between infernce time and model accuracy. </li>

In [6]:
df,styler_df=kgnet.KGMeta_Governer.getGMLTaskModelsBasicInfoByID(taskId)
styler_df

,Property,Model ID,GNN Method,Training Memory GB.,Training Time Sec.,Test Acc Score,Inference Time,Date Created
0,Model #1,83,G_SAINT,0.927505,538.690000,94,0,"21/08/2023, 15:38:02"
1,Model #2,85,G_SAINT,1.025944,379.460000,72,0,"21/08/2023, 15:57:26"
2,Model #3,82,G_SAINT,1.054363,516.560000,93,0,"21/08/2023, 14:29:39"
3,Model #4,91,G_SAINT,0.995323,285.370000,72,0,"21/08/2023, 18:49:41"
4,Model #5,96,ShaDowSAINT,5.985710,462.340000,23,0,"23/08/2023, 17:59:59"
5,Model #6,98,RGCN,2.652660,27.270000,23,-1,"23/08/2023, 21:17:38"
6,Model #7,77,G_SAINT,0.991444,471.590000,94,0,"19/08/2023, 14:46:30"
7,Model #8,99,RGCN,0.963436,45.160000,23,-1,"24/08/2023, 14:29:32"
8,Model #9,94,G_SAINT,1.219990,784.470000,98,0,"23/08/2023, 13:47:37"
9,Model #10,95,G_SAINT,3.330479,68.340000,31,3,"23/08/2023, 16:09:08"


## Inference Pipeline
write the SPARQL-ML inference query then execute it. KGNet will choose the model that match your task.

In [7]:
query = """
            prefix dblp2022:<https://dblp.org/rdf/schema#>
            prefix kgnet:<http://kgnet/>
            select ?Publication ?Title ?dblp_Venue ?Pred_Venue
            from <https://dblp2022.org>
            where
            {
            ?Publication a dblp2022:Publication .
            ?Publication ?NodeClassifier ?Pred_Venue .            
            ?Publication  dblp2022:publishedIn ?dblp_Venue .
            ?Publication  dblp2022:title ?Title .
            ?Publication  dblp2022:numberOfCreators "3" .            
            ?NodeClassifier a <kgnet:types/NodeClassifier>.
            ?NodeClassifier <kgnet:targetNode> dblp2022:Publication.
            ?NodeClassifier <kgnet:labelNode> dblp2022:publishedIn_Obj.
            }
            limit 1000
        """


In [8]:
import datetime
startT = datetime.datetime.now()
res_df,inferenceQueryMetadata=kgnet.executeSPARQLMLInferenceQuery(query)
print("The task selected model ID is:",inferenceQueryMetadata["model_id"])
print('time=',datetime.datetime.now()-startT)

The task selected model ID is: 94
time= 0:00:20.990226


In [9]:
highlighted_df=res_df.style.apply(kgnet.utils.highlightDiferrentRowValues, subset=['dblp_Venue','Pred_Venue'], axis=1)
highlighted_df

,Publication,Title,dblp_Venue,Pred_Venue
0,"""https://dblp.org/rec/conf/ctrsa/CidIR22""","""FASTA - A Stream Cipher for Fast FHE Evaluation.""","""CT-RSA""","""CT-RSA"""
1,"""https://dblp.org/rec/conf/padl/AriasCG22""","""Towards Dynamic Consistency Checking in Goal-Directed Predicate Answer Set Programming.""","""PADL""","""PADL"""
2,"""https://dblp.org/rec/conf/padl/BellusciMR22""","""Modelling the Outlier Detection Problem in ASP(Q).""","""""","""PADL"""
3,"""https://dblp.org/rec/conf/padl/BistarelliMT22""","""Timed Concurrent Language for Argumentation: An Interleaving Approach.""","""""","""PADL"""
4,"""https://dblp.org/rec/conf/padl/BurghardtHG22""","""Functional Programming on Top of SQL Engines.""","""PADL""","""PADL"""
5,"""https://dblp.org/rec/conf/padl/EvansFW22""","""sf CircuitFlow: A Domain Specific Language for Dataflow Programming.""","""PADL""","""PADL"""
6,"""https://dblp.org/rec/conf/padl/FaberMC22""","""Determining Action Reversibility in STRIPS Using Answer Set Programming with Quantifiers.""","""PADL""","""PADL"""
7,"""https://dblp.org/rec/conf/blockchain/Ako-NaiMT21""","""Artificial Intelligence Decision and Validation Powered Smart Contract for Open Learning Content Creation.""","""BLOCKCHAIN""","""BLOCKCHAIN"""
8,"""https://dblp.org/rec/conf/blockchain/DongMW21""","""Blockchain-Enabled Next Generation Access Control.""","""BLOCKCHAIN""","""BLOCKCHAIN"""
9,"""https://dblp.org/rec/conf/blockchain/GointBD21""","""Establish Trust for Sharing Data for Smart Territories Thanks to Consents Notarized by Blockchain.""","""BLOCKCHAIN""","""BLOCKCHAIN"""
